In [1]:
%load_ext pycodestyle_magic
%flake8_on

# Формат XML

Мы будем пользоваться модулем **ElementTree**, входящим в стандартный пакет xml. Этот модуль позволит нам ходить по дереву XML и смотреть, что находится в каждом его узле, начиная с корня и заканчивая листьями. Импортируем модуль:

In [2]:
import xml.etree.ElementTree as ET

In [3]:
tree = ET.parse('./data/menu.xml')
root = tree.getroot()

In [4]:
print(root)

<Element 'menu' at 0x7f94eecd4290>


Как посмотреть список детей (потомков) этого узла? С помощью метода list(root):

In [5]:
list(root)

[<Element 'dish' at 0x7f94eecd43b0>, <Element 'dish' at 0x7f94eecd46b0>]

Метод возвращает список. Если у узла нет детей, то он вернет пустой список [].

In [6]:
list(root)[0]

<Element 'dish' at 0x7f94eecd43b0>

In [7]:
root[0]

<Element 'dish' at 0x7f94eecd43b0>

Ранее мы видели, что у узлов могут быть параметры (атрибуты). Например, у узлов <dish> мы видели атрибут name. Мы можем обратиться к атрибутам объекта с помощью команды attrib. Здесь может возникнуть небольшая путаница, потому что мы говорим о двух разных определениях слова "атрибут" в нашем контексте:

1. Атрибут у тега (как name у <dish>).
2. Атрибут объекта (переменная класса). 
    
В данном случае мы берем объект типа ElementTree.Element, у которого есть атрибут attrib. В этом атрибуте объекта хранится словарь с атрибутами данного узла.

Это значит, что если мы обратимся к attrib, то нам вернется словарь, в котором ключами будут названия атрибутов, а значениями — соответствующие значения. Чтобы стало яснее, давайте посмотрим на атрибуты у <dish>:

In [8]:
root[0].attrib

{'name': 'Кура'}

## Чтение значений узлов

В узлах XML часто хранятся количественные показатели. Эти показатели хранятся в виде текста, и прочитать их можно, обратившись к атрибуту text у соответствующего объекта типа ElementTree.Element. Например, возьмем узел price:

In [9]:
root[0][0]

<Element 'price' at 0x7f94eecd44d0>

Теперь, прочитаем значение этого узла:

In [10]:
root[0][0].text

'40'

Заметьте, что значение '40' представлено строкой. Все значения в XML хранятся как строки, поэтому преобразовывать их к нужному типу вам нужно самим. Например, в данном случае можно обернуть значение стоимости в int() или float().

Если вы хотите прочитать название тега для конкретного узла, можно использовать атрибут tag.

Какое значение тега у узла root[0][1]?

In [11]:
root[0][1].tag

'weight'

## Использование циклов

In [12]:
for elem in root:
    for subelem in elem:
        print(elem.attrib['name'], subelem.tag, subelem.text)
    print()

Кура price 40
Кура weight 300
Кура class Мясо

Греча price 20
Греча weight 200
Греча class Крупа



Посчитайте, сколько детей у первого объекта dish. Для этого используйте метод len():

In [13]:
len(root[0])

3

Для дальнейшей работы нам необходимо установить библиотеку xmljson, для этого выполните команду в Jupyter Notebook: **!pip install xmljson**

In [14]:
!pip install xmljson

## Превращение XML в pd.DataFrame

In [18]:
import pandas as pd

In [19]:
df_index = ['name', 'price', 'weight', 'class']
df = pd.DataFrame(columns=df_index)

for elem in root:
    elements = [elem.get('name'), elem[0].text, elem[1].text, elem[2].text]
    df = df.append(pd.Series(elements, index=df_index), ignore_index=True)

In [20]:
df

,name,price,weight,class
0,Кура,40,300,Мясо
1,Греча,20,200,Крупа


## Превращение XML в JSON

In [22]:
import xmljson
xmljson.parker.data(root)

OrderedDict([('dish',
              [OrderedDict([('price', 40),
                            ('weight', 300),
                            ('class', 'Мясо')]),
               OrderedDict([('price', 20),
                            ('weight', 200),
                            ('class', 'Крупа')])])])

Если вы захотите использовать другое соглашение, просто замените в коде название одного соглашения на название другого. Например,

In [23]:
xmljson.gdata.data(root)

OrderedDict([('menu',
              OrderedDict([('dish',
                            [OrderedDict([('name', 'Кура'),
                                          ('price', OrderedDict([('$t', 40)])),
                                          ('weight',
                                           OrderedDict([('$t', 300)])),
                                          ('class',
                                           OrderedDict([('$t', 'Мясо')]))]),
                             OrderedDict([('name', 'Греча'),
                                          ('price', OrderedDict([('$t', 20)])),
                                          ('weight',
                                           OrderedDict([('$t', 200)])),
                                          ('class',
                                           OrderedDict([('$t',
                                                         'Крупа')]))])])]))])

Как превратить JSON обратно в XML? Использовать метод etree() у класса-соглашения:

In [24]:
parker_json = xmljson.parker.data(root)
back_to_xml = xmljson.parker.etree(parker_json)

## Запись в XML-файл

Осталось самое последнее — научиться создавать свои XML-файлы. Вероятно, это почти никогда не пригодится вам, но все же такая ситуация может возникнуть. Чтобы создать корень дерева, нужно использовать метод Element() из класса ElementTree:

In [27]:
new_root = ET.Element('menu')

In [28]:
dish_1 = ET.SubElement(new_root, 'dish')
dish_2 = ET.SubElement(new_root, 'dish')
list(new_root)

[<Element 'dish' at 0x7f94f1dbec50>, <Element 'dish' at 0x7f94f1dbeb30>]

Атрибуты можно добавить с помощью метода set(), передав первым параметром название атрибута, а вторым — его значение:

In [29]:
dish_1.set('name', 'Кура')

Значение тега можно задать через уже знакомый вам параметр text:

In [30]:
dish_1.text = 'Белок'

Теперь запишем получившийся XML-файл на диск, используя стандартные средства Python. Сначала превратим объект типа ElementTree.Element в строку (str) c помощью метода tostring(), передав наше новое дерево как аргумент:

In [31]:
new_root_string = ET.tostring(new_root)

In [35]:
ET.ElementTree(new_root).write('./data/new_menu_good.xml', encoding="utf-8")